In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
import time

In [ ]:
def find_reporter_yahoo(soup):
    return soup.find('span',class_="provider-link").text
def find_report_time_yahoo(soup):
    return soup.find('time').text.replace('年','-').replace('月','-').replace('日','')
def find_text_yahoo(soup):
    try:
        texts=soup.find('article',itemprop="articleBody").text
    except:
        items=soup.find('article').find_all('p')
        texts=''
        for item in items:
            texts+=item.text
    return texts

In [ ]:
data_url=[]
data_title=[]
keywords=['颱風','天氣']#補上

In [ ]:
for k in keywords[1:2]:
    count=0
    while True:
        url_origin='https://tw.news.yahoo.com/_td-news/api/resource/NewsSearchService;query='+k+';offset='
        url=url_origin+str(count*10) 
        #print(url)
        count+=1
        
        resp=requests.get(url,headers=headers)
        if len(resp.json())!=0:
            for i in range(0,len(resp.json())):
                title=resp.json()[i]['title']
                url='https://tw.news.yahoo.com'+resp.json()[i]['url']
                data_title.append(title)
                data_url.append(url)
                print('標題:'+title)
                print('網址:'+url)
        else:
            print("can't crawl")
            break

In [ ]:
if len(data_title)==len(data_url):
    data=pd.DataFrame()
    data['title']=data_title
    data['url']=data_url
    data['website']='Yahoo'
    print("make dataframe")

In [ ]:
data_reporter=[]
data_text=[]
data_time=[]


count=0
for i in data['url']:
    try:
        resp=requests.get(i,headers=headers)
        resp.encoding='utf-8'
        items=BeautifulSoup(resp.text,"html5lib")
        #items=soup.find_all("div",class_="inner")
        try:
            x=find_reporter_yahoo(items)
            data_reporter.append(x)
            print(x)
        except:
            data_reporter.append("None")
        try:
            y=find_text_yahoo(items)
            data_text.append(y)
            print(y)
        except:
            data_text.append("None")
        try:
            z=find_report_time_yahoo(items)
            data_time.append(z)
            print(z)
        except:
        
            data_time.append("None")
    except:
        print("fail,全部為None")
        data_text.append("None")
        data_time.append("None")
        data_reporter.append("None")
    count+=1
    print("目前執行次數 %d" % count)
    print("目前共有幾筆文章資料 %d" % len(data_text))
    print("目前共有幾筆時間資料 %d" % len(data_time))

In [ ]:
data['reporter']=data_reporter
data['time']=data_time
data['text']=data_text

In [ ]:
#-------留言及回覆部分-------

In [6]:
def change_timestamp(comment_time):
    timeStamp = comment_time
    timeArray = time.localtime(timeStamp)
    return time.strftime("%Y-%m-%d", timeArray)

In [4]:
#獲取文章的context_id，用於留言api的網址內
#url='https://tw.news.yahoo.com/幫助台灣人回台有錯嗎-徐正文8點聲明-032829693.html'
url='https://tw.news.yahoo.com/%E5%AD%B8%E7%94%9F%E6%9F%93%E7%96%AB-%E5%8F%B0%E5%B8%AB%E5%A4%A7%E5%85%A8%E6%A0%A146-17%E6%94%B9%E6%8E%A1%E9%81%A0%E8%B7%9D%E6%95%99%E5%AD%B8-102519711.html'
resp=requests.get(url)
soup=BeautifulSoup(resp.text,"html5lib")
context_id=soup.find('article').find('a')['data-ylk'].split('g:')[1].split(';',1)[0]
context_id

'ed8174f2-6c99-3967-b9a9-642c8bc5288c'

In [7]:
data_comment_reporter=[]
data_comment_text=[]
data_comment_time=[]

num=-1
count=0
data_message_id=[]
while True:
    url='https://tw.news.yahoo.com/_td/api/resource/canvass.getMessageListForContext_ns;apiVersion=v1;context='+context_id+';count=30;index=v%3D1%3As%3Dpopular%3Asl%3D1581042682%3Aoff%3D'+str(num)+';lang=zh-Hant-TW;namespace=yahoo_content;oauthConsumerKey=frontpage.oauth.canvassKey;oauthConsumerSecret=frontpage.oauth.canvassSecret;rankingProfile=;region=TW;sortBy=popular;userActivity=true?bkt=news-TW-zh-Hant-TW-def'
    resp=requests.get(url,headers=headers).json()
    if len(resp['canvassMessages'])!=0:
        for comment in range(0,len(resp['canvassMessages'])):
            print('留言者:'+resp['canvassMessages'][comment]['meta']['author']['nickname'])#留言者
            data_comment_reporter.append(resp['canvassMessages'][comment]['meta']['author']['nickname'])
            comment_time=change_timestamp(resp['canvassMessages'][comment]['meta']['createdAt'])
            print('留言時間:'+str(comment_time))#留言時間
            data_comment_time.append(comment_time)
            print('留言內容:'+resp['canvassMessages'][comment]['details']['userText'])#留言內容
            data_comment_text.append(resp['canvassMessages'][comment]['details']['userText'])
            if resp['canvassMessages'][comment]['reactionStats']['replyCount']>0:
                message_id=resp['canvassMessages'][comment]['messageId']
                data_message_id.append(message_id)
            count+=1
            print('-----------')
        num+=30
    else:
        break
        
for i_id in data_message_id:
    url='https://tw.news.yahoo.com/_td/api/resource/canvass.getReplies_ns;action=showNext;apiVersion=v1;context='+context_id+';count=30;lang=zh-Hant-TW;messageId='+i_id+';namespace=yahoo_content;oauthConsumerKey=frontpage.oauth.canvassKey;oauthConsumerSecret=frontpage.oauth.canvassSecret;region=TW;sortBy=createdAt;tags=?bkt=TW-Non-NCP-stream-applet-control'
    resp=requests.get(url).json()
    for message in range(0,len(resp)):
        print('留言回覆者:'+resp[message]['meta']['author']['nickname'])
        data_comment_reporter.append(resp[message]['meta']['author']['nickname'])
        print('留言回覆時間:'+change_timestamp(resp[message]['meta']['createdAt']))
        data_comment_time.append(change_timestamp(resp[message]['meta']['createdAt']))
        print('留言回覆內容:'+resp[message]['details']['userText'])
        data_comment_text.append(resp[message]['details']['userText'])
        print('-------------')
        count+=1

留言者:tai
留言時間:2020-04-01
留言內容:書什麼時候都可以念，命只有一條
-----------
留言者:Mikel
留言時間:2020-04-01
留言內容:果斷 比起時鐘 除開對中國政治防疫 對歐美防疫憂柔寡斷 破口一大堆 置台灣於險境
-----------
留言者:change
留言時間:2020-04-01
留言內容:美術系的怎遠距
-----------
留言者:******
留言時間:2020-04-01
留言內容:可以公布哪位了！還有行蹤接觸史
-----------
留言者:游來游去
留言時間:2020-04-01
留言內容:這是對的。
-----------
留言者:tipi
留言時間:2020-04-01
留言內容:台師大同學，這段期間辛苦了，
最近台北市一部分的防疫靠你們的自律了，台北市小市民謝謝您
-----------
留言者:vampire
留言時間:2020-04-01
留言內容:各自停有甚麼用，就像只封大陸，不封歐美一樣，病例一直狂增
-----------
留言者:2311
留言時間:2020-04-01
留言內容:全國應有限度的禁外出
-----------
留言者:小食堂堂主
留言時間:2020-04-01
留言內容:4/6 之前 盡量吸收病毒 ???
-----------
留言者:00哈00
留言時間:2020-04-02
留言內容:誠時鐘在上半場被捧成像神,但下半場一團糟,如果台灣疫情變嚴重,,,輸光頭會躲開,時鐘會變成萬惡的罪人,,
-----------
留言者:Hsiao-Tai
留言時間:2020-04-01
留言內容:昨日就應該立即宣佈全校遠距離上課了，還拖到今日才宣布，真是防疫慢半拍。
-----------
留言者:Oursmon
留言時間:2020-04-02
留言內容:再不公布足跡，兩個星期後應該會爆發吧，因為大家都出去玩了，公布的話大家會比較節制一點。
-----------
留言者:Ppycjqq
留言時間:2020-04-01
留言內容:那個騎腳踏車跌倒然後踹計程車的教授要注意身體蛤~
-----------
留言者:刪留言下地獄
留言時間:2020-04-01
留言內容:四面環海的小島防疫做成這樣，還不停課，全部停課是不是防疫破功了呢，每天吹牛造神，把學生當試驗品！草菅人命！
-

In [ ]:
if len(data_comment_reporter)==len(data_comment_text):
    data_comment=pd.DataFrame()
    data_comment['commenter']=data_comment_reporter
    data_comment['comment_time']=data_comment_time
    data_comment['text']=data_comment_text
    print("make dataframe")

In [ ]:
data_comment

In [8]:
len(data_comment_text)

66

In [26]:
url='https://tw.news.yahoo.com/_td/api/resource/canvass.getMessageListForContext_ns;apiVersion=v1;context=ed8174f2-6c99-3967-b9a9-642c8bc5288c;count=30;lang=zh-Hant-TW;namespace=yahoo_content;oauthConsumerKey=frontpage.oauth.canvassKey;oauthConsumerSecret=frontpage.oauth.canvassSecret;rankingProfile=;sortBy=popular'

data_comment_reporter=[]
data_comment_text=[]
data_comment_time=[]
data_message_id=[]


while True:
    resp=requests.get(url).json()

    if len(resp['canvassMessages'])!=0:
        for comment in range(0,len(resp['canvassMessages'])):
            print('留言者:'+resp['canvassMessages'][comment]['meta']['author']['nickname'])#留言者
            data_comment_reporter.append(resp['canvassMessages'][comment]['meta']['author']['nickname'])
            comment_time=change_timestamp(resp['canvassMessages'][comment]['meta']['createdAt'])
            print('留言時間:'+str(comment_time))#留言時間
            data_comment_time.append(comment_time)
            print('留言內容:'+resp['canvassMessages'][comment]['details']['userText'])#留言內容
            print('comment_id:'+resp['canvassMessages'][comment]["messageId"])
            print('like_num:'+str(resp['canvassMessages'][comment]["reactionStats"]["upVoteCount"]))
            data_comment_text.append(resp['canvassMessages'][comment]['details']['userText'])
            if resp['canvassMessages'][comment]['reactionStats']['replyCount']>0:
                message_id=resp['canvassMessages'][comment]['messageId']
                data_message_id.append(message_id)
            count+=1
            print('-----------')
        if resp["nextIndex"] != None:
            if ';index=' in url:
                url=url.split(';index=')[0]
            url=url+';index='+resp["nextIndex"]
    else:
        break

留言者:tai
留言時間:2020-04-01
留言內容:書什麼時候都可以念，命只有一條
comment_id:bac91a4a-e230-4dff-a77b-974ab45682e0
like_num:24
-----------
留言者:Mikel
留言時間:2020-04-01
留言內容:果斷 比起時鐘 除開對中國政治防疫 對歐美防疫憂柔寡斷 破口一大堆 置台灣於險境
comment_id:92758d9e-27e4-4aec-952b-c4ee46dc9301
like_num:58
-----------
留言者:change
留言時間:2020-04-01
留言內容:美術系的怎遠距
comment_id:a416885a-d94d-4a88-99d7-50b46486dfdf
like_num:17
-----------
留言者:******
留言時間:2020-04-01
留言內容:可以公布哪位了！還有行蹤接觸史
comment_id:2793ab4a-2a0b-4df6-aa29-c7977d306771
like_num:19
-----------
留言者:游來游去
留言時間:2020-04-01
留言內容:這是對的。
comment_id:e2d16f83-68c2-46a5-abd6-769350a45613
like_num:19
-----------
留言者:tipi
留言時間:2020-04-01
留言內容:台師大同學，這段期間辛苦了，
最近台北市一部分的防疫靠你們的自律了，台北市小市民謝謝您
comment_id:a6bd5d01-11c9-4349-b576-25b0faf89746
like_num:17
-----------
留言者:vampire
留言時間:2020-04-01
留言內容:各自停有甚麼用，就像只封大陸，不封歐美一樣，病例一直狂增
comment_id:23d81528-ecac-4082-adc3-1a4c71890c27
like_num:25
-----------
留言者:2311
留言時間:2020-04-01
留言內容:全國應有限度的禁外出
comment_id:de6dedd8-1c4e-4844-8cc2-60476a3ba73f
like_num:7
-----------
留言者:00哈

In [12]:
resp["nextIndex"]

'v=1:s=popular:sl=1585785462:off=29'

In [22]:
len(data_comment_reporter)

49

In [29]:
import json
url='https://tw.news.yahoo.com/_td/api/resource/canvass.getReplies_ns;action=showNext;apiVersion=v1;context=ed8174f2-6c99-3967-b9a9-642c8bc5288c;count=30;index=null;lang=zh-Hant-TW;messageId=a416885a-d94d-4a88-99d7-50b46486dfdf;namespace=yahoo_content;oauthConsumerKey=frontpage.oauth.canvassKey;oauthConsumerSecret=frontpage.oauth.canvassSecret'
resp=requests.get(url)
data=json.loads(resp.text)

In [35]:
for i in range(0,len(data)):
    print('reply_id:'+data[i]['replyId'])
    print('author_id'+data[i]['meta']['author']['guid'])
    print('author_name'+data[i]['meta']['author']['nickname'])
    print('context'+data[i]['details']['userText'])
    

396714dc-6a3d-4dbe-afea-3d26add0d00f
17c3a7a0-0331-4410-8602-922dcedc9832
ef0575ab-0adf-4faf-a4ec-efbe66509671
885dd2a9-9ff0-4beb-9f14-0315b4f28473
9308e08c-830c-4b0d-bde0-2f1715ddd0e2
4bce1f8c-fc68-48d1-b5e5-a5ba42bbc732
9716a053-452e-469f-a414-a36dad2256ee
c229f2af-c4ce-4f85-82f0-77ab4b387047
4c0d31d7-4394-4fc1-97a5-83e03aad85e3
b554fb18-0497-4820-a1fd-1763c0aaa47e
b5500a61-b111-4aff-a934-0114717c979d
840e13b1-d98c-488a-a479-ce486a1953b0
b61bc9fc-ccfd-4c73-983a-01c7b93214c1


In [48]:
data[1]['meta']['createdAt']

{'type': 'TEXT',
 'author': {'guid': 'LQKNKBNQP7QUIS3TGXRGG4WZJ4',
  'userType': 'YAHOO_USER',
  'nickname': 'ｆｙｏｕｈｉｐｙ７ｔ',
  'image': {'url': 'https://s.yimg.com/ny/api/res/1.2/XgWWX2OziDzG6y.l8L29WQ--~A/YXBwaWQ9aGlnaGxhbmRlcjt3PTgwO2g9ODA7cT04MA--/https://s.yimg.com/gq/1756/40147164326_afa8df_o.jpg',
   'height': 40,
   'width': 40},
  'userCategory': 'REGULAR_USER'},
 'createdAt': 1585748884,
 'updatedAt': 1585748884,
 'contextInfo': {'url': 'https://tw.news.yahoo.com/%E5%AD%B8%E7%94%9F%E6%9F%93%E7%96%AB-%E5%8F%B0%E5%B8%AB%E5%A4%A7%E5%85%A8%E6%A0%A146-17%E6%94%B9%E6%8E%A1%E9%81%A0%E8%B7%9D%E6%95%99%E5%AD%B8-102519711.html',
  'displayText': '學生染疫 台師大：全校4/6-17改採遠距教學'},
 'scoreAlgo': 'canvassPerspectiveAndCommunityV3AndRelevanceProfile',
 'visibility': 'PUBLIC',
 'locale': {'region': 'TW', 'lang': 'zh-Hant-TW'},
 'mentions': [],
 'messageEntity': 'COMMENT'}